# Imports

In [40]:
import requests
import json
import pandas as pd
import time
from time import sleep
import random
from random import randint
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', None)


# Extract info from website :

In [41]:
%%time

pages = []
for i in range(1,5000):

    response = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params = {
        'currency_code':'EUR',
        'grape_filter':'varietal',
        'min_rating':'3.6',
        'order_by':'ratings_average',
        'order':'desc',
        'wine_type_ids[]':'3',
        'page':i,
        'language':'en'
    },
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }
)

    #store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,2000)
    sleep(wait_time/1000)

CPU times: user 54.4 s, sys: 17.7 s, total: 1min 12s
Wall time: 2h 16min 28s


In [42]:
len(pages)

4999

In [43]:
# Nr. of wines
response.json()['explore_vintage']['records_matched']

2099

In [44]:
response.json()['explore_vintage']['matches'][0].keys()

dict_keys(['vintage', 'price', 'prices'])

## Vintage

In [45]:
all_wines = []
vintage = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        vintage.append(all_wines[i][j]['vintage'])

In [46]:
vintage_df = pd.DataFrame(vintage)
vintage_df.head(3)

,id,seo_name,name,statistics,image,wine,year,grapes,has_valid_ratings,top_list_rankings
0,2152428,bollinger-vieilles-vignes-francaises-blanc-de-...,Bollinger Vieilles Vignes Françaises Blanc de ...,"{'status': 'Normal', 'ratings_count': 118, 'ra...",{'location': '//images.vivino.com/thumbs/4rhiQ...,"{'id': 18938, 'name': 'Vieilles Vignes Françai...",2007,None,True,NaN
1,108802,krug-clos-du-mesnil-blanc-de-blancs-brut-champ...,Krug Clos du Mesnil Blanc de Blancs Brut Champ...,"{'status': 'Normal', 'ratings_count': 45, 'rat...",{'location': '//images.vivino.com/thumbs/xtWMP...,"{'id': 79635, 'name': 'Clos du Mesnil Blanc de...",1985,None,True,NaN
2,2669463,krug-clos-d-ambonnay-blanc-de-noirs-brut-champ...,Krug Clos d'Ambonnay Blanc de Noirs Brut Champ...,"{'status': 'Normal', 'ratings_count': 140, 'ra...",{'location': '//images.vivino.com/thumbs/DPq0a...,"{'id': 1220504, 'name': 'Clos d'Ambonnay Blanc...",1995,None,True,NaN


In [47]:
vintage_df['id'].nunique()

2023

In [48]:
#vintage_df['top_list_rankings'].value_counts(dropna = False)

#### Remove unnecessary columns

 - 'statistics' and 'wine' info will be extracted separetely, therefore I will drop these columns as well;
 - 'image' and 'top_list_rankings' are not required for the model
 - 'grapes' only has None values. This info is captured under wines section
 - 'has_valid_ratings' is always 'True'

In [49]:
vintage_df = vintage_df.drop(['statistics','image','wine','grapes','has_valid_ratings','top_list_rankings'], axis = 1)
vintage_df

,id,seo_name,name,year
0,2152428,bollinger-vieilles-vignes-francaises-blanc-de-...,Bollinger Vieilles Vignes Françaises Blanc de ...,2007
1,108802,krug-clos-du-mesnil-blanc-de-blancs-brut-champ...,Krug Clos du Mesnil Blanc de Blancs Brut Champ...,1985
2,2669463,krug-clos-d-ambonnay-blanc-de-noirs-brut-champ...,Krug Clos d'Ambonnay Blanc de Noirs Brut Champ...,1995
3,1161459,salon-champagne-grand-cru-le-mesnil-sur-oger-l...,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...,1996
4,1324060,krug-clos-d-ambonnay-blanc-de-noirs-brut-champ...,Krug Clos d'Ambonnay Blanc de Noirs Brut Champ...,1998
...,...,...,...,...
124970,3416176,castell-d-age-cava-anne-marie-brut-nature-rese...,Castell d'Age Cava Anne Marie Brut Nature Rese...,N.V.
124971,143976990,andre-et-michel-quenard-cremant-de-savoie-extr...,André et Michel Quenard Crémant de Savoie Extr...,N.V.
124972,2277886,us-chronic-spritz-and-giggles-nv,Chronic Spritz and Giggles N.V.,N.V.
124973,2558045,val-d-oca-prosecco-treviso-frizzante-nv,Val d'Oca Prosecco Treviso Frizzante N.V.,N.V.


## Statistics

In [50]:
all_wines = []
statistics = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        statistics.append(all_wines[i][j]['vintage']['statistics'])

In [51]:
statistics_df = pd.DataFrame(statistics)
statistics_df.head(3)

,status,ratings_count,ratings_average,labels_count,wine_ratings_count,wine_ratings_average,wine_status
0,Normal,118,4.8,269,1161,4.7,Normal
1,Normal,45,4.9,163,2385,4.7,Normal
2,Normal,140,4.9,366,826,4.8,Normal


In [52]:
len(statistics_df['status'])

124975

#### Remove unnecessary columns

In [53]:
# for col in statistics_df.columns:
#     #print(customer_df[col].unique)
#     print('Name: {}'.format(statistics_df[col].name))    # column name
#     print('Type: {}'.format(statistics_df[col].dtype))   # column type
#     print('Unique values: {}'.format(len(statistics_df[col].unique())))   # column unique values
#     print('% Unique values: {}'.format(round(len(statistics_df[col].unique())/len(statistics_df)*100,1)))  # percentage
#     print('\n----------')
    

In [54]:
statistics_df = statistics_df.drop(['status','labels_count','wine_ratings_count','wine_ratings_average','wine_status'], axis = 1)
statistics_df

,ratings_count,ratings_average
0,118,4.8
1,45,4.9
2,140,4.9
3,992,4.8
4,183,4.8
...,...,...
124970,636,3.6
124971,433,3.6
124972,641,3.6
124973,422,3.6


## Wine info

In [55]:
all_wines = []
wine = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        wine.append(all_wines[i][j]['vintage']['wine'])

In [56]:
wine_df = pd.DataFrame(wine)

wine_df.columns = 'wine_' + wine_df.columns
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True
1,79635,Clos du Mesnil Blanc de Blancs Brut Champagne,clos-du-mesnil-blanc-de-blancs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.550288, 'fizziness...","{'status': 'Normal', 'ratings_count': 2385, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True
2,1220504,Clos d'Ambonnay Blanc de Noirs Brut Champagne,clos-d-ambonnay-blanc-de-noirs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.605912, 'fizziness...","{'status': 'Normal', 'ratings_count': 826, 'ra...","{'id': 50, 'seo_name': 'french-champagne', 're...",True


### wine_region

In [57]:
#Flat columns
wine_region = pd.json_normalize(wine_df['wine_region'])
wine_region.columns = 'wine_region_' + wine_region.columns
wine_region.shape

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_region], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_region'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 33)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN
1,79635,Clos du Mesnil Blanc de Blancs Brut Champagne,clos-du-mesnil-blanc-de-blancs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.550288, 'fizziness...","{'status': 'Normal', 'ratings_count': 2385, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN
2,1220504,Clos d'Ambonnay Blanc de Noirs Brut Champagne,clos-d-ambonnay-blanc-de-noirs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.605912, 'fizziness...","{'status': 'Normal', 'ratings_count': 826, 'ra...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN


### wine_winery

In [58]:
#Flat columns
wine_winery = pd.json_normalize(wine_df['wine_winery'])
wine_winery.columns = 'wine_winery_' + wine_winery.columns
# print(wine_winery.shape)
# wine_winery

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_winery], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 42)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,7747,Bollinger,bollinger,0,//images.vivino.com/wineries/backgrounds/cR1pN...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,NaN
1,79635,Clos du Mesnil Blanc de Blancs Brut Champagne,clos-du-mesnil-blanc-de-blancs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.550288, 'fizziness...","{'status': 'Normal', 'ratings_count': 2385, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,1337,Krug,krug,0,//images.vivino.com/wineries/backgrounds/H7zrN...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,NaN
2,1220504,Clos d'Ambonnay Blanc de Noirs Brut Champagne,clos-d-ambonnay-blanc-de-noirs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.605912, 'fizziness...","{'status': 'Normal', 'ratings_count': 826, 'ra...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,1337,Krug,krug,0,//images.vivino.com/wineries/backgrounds/H7zrN...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,NaN


### wine_taste

In [59]:
#Flat columns
wine_taste = pd.json_normalize(wine_df['wine_taste'])
wine_taste.columns = 'wine_taste_' + wine_taste.columns
#print(wine_taste.shape)
#wine_taste

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_taste], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 51)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_flavor,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,7747,Bollinger,bollinger,0,//images.vivino.com/wineries/backgrounds/cR1pN...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,NaN,"[{'group': 'oak', 'stats': {'count': 73, 'scor...",4.354505,4.245162,4.184379,NaN,NaN,166.0,71.0,NaN
1,79635,Clos du Mesnil Blanc de Blancs Brut Champagne,clos-du-mesnil-blanc-de-blancs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.550288, 'fizziness...","{'status': 'Normal', 'ratings_count': 2385, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,1337,Krug,krug,0,//images.vivino.com/wineries/backgrounds/H7zrN...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,//thumbs.vivino.com/winery_backgrounds/H7zrN96...,NaN,"[{'group': 'non_oak', 'stats': {'count': 106, ...",4.550288,4.215910,3.968117,NaN,NaN,310.0,131.0,NaN
2,1220504,Clos d'Ambonnay Blanc de Noirs Brut Champagne,clos-d-ambonnay-blanc-de-noirs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.605912, 'fizziness...","{'status': 'Normal', 'ratings_count': 826, 'ra...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//t

#### wine_taste_flavor

In [60]:
#Flat columns
wine_flavor = pd.json_normalize(wine_df['wine_taste_flavor'])


# #Flat the first 4 columns from flavor table. This are most voted tastes by users
wine_flavor_1 = pd.json_normalize(wine_flavor[0])
wine_flavor_1.columns = 'wine_taste_flavor_1' + wine_flavor_1.columns
wine_flavor_2 = pd.json_normalize(wine_flavor[1])
wine_flavor_2.columns = 'wine_taste_flavor_2' + wine_flavor_2.columns
wine_flavor_3 = pd.json_normalize(wine_flavor[2])
wine_flavor_3.columns = 'wine_taste_flavor_3' + wine_flavor_3.columns
wine_flavor_4 = pd.json_normalize(wine_flavor[3])
wine_flavor_4.columns = 'wine_taste_flavor_4' + wine_flavor_4.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_flavor_1, wine_flavor_2, wine_flavor_3, wine_flavor_4], axis=1)
wine_df = wine_df.drop('wine_taste_flavor', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1primary_keywords,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,7747,Bollinger,bollinger,0,//images.vivino.com/wineries/backgrounds/cR1pN...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,NaN,4.354505,4.245162,4.184379,NaN,NaN,166.0,71.0,NaN,oak,"[{'id': 70, 'name': 'butter', 'count': 12}, {'...","[{'id': 62, 'name': 'brioche', 'count': 32}, {...",73.0,7652.0,non_oak,"[{'id': 62, 'name': 'brioche', 'count': 32}, {...","[{'id': 434, 'name': 'vanilla', 'count': 12}, ...",69.0,5039.0,tree_fruit,"[{'id': 9, 'name': 'apple', 'count': 27}, {'id...",55.0,7200.0,NaN,microbio,"[{'id': 123, 'name': 'cream', 'count': 18}, {'...","[{'id': 417, 'name': 'toast', 'count': 16}, {'...",53.0,4488.0
1,79635,Clos du Mesnil Blanc de Blancs Brut Champagne,clos-du-mesnil-blanc-de-blancs-brut-champagne,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 1337, 'name': 'Krug', 'seo_name': 'krug...","{'structure': {'acidity': 4.550288, 'fizziness...","{'status': 'Normal', 'ratings_count': 2385, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDA

#### wine_taste_flavor_keywords

In [61]:
wine_taste_keyword1 = pd.json_normalize(wine_df['wine_taste_flavor_1primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword1_1 = pd.json_normalize(wine_taste_keyword1[0])
wine_taste_keyword1_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword1_1.columns
wine_taste_keyword1_2 = pd.json_normalize(wine_taste_keyword1[1])
wine_taste_keyword1_2.columns = 'wine_taste_flavor_2primary_keywords' + wine_taste_keyword1_2.columns
wine_taste_keyword1_3 = pd.json_normalize(wine_taste_keyword1[2])
wine_taste_keyword1_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword1_3.columns
wine_taste_keyword1_4 = pd.json_normalize(wine_taste_keyword1[3])
wine_taste_keyword1_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword1_4.columns
wine_taste_keyword1_5 = pd.json_normalize(wine_taste_keyword1[4])
wine_taste_keyword1_5.columns = 'wine_taste_flavor_5primary_keywords' + wine_taste_keyword1_5.columns



#combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword1_1, wine_taste_keyword1_2, wine_taste_keyword1_3, wine_taste_keyword1_4, wine_taste_keyword1_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_1primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,7747,Bollinger,bollinger,0,//images.vivino.com/wineries/backgrounds/cR1pN...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,NaN,4.354505,4.245162,4.184379,NaN,NaN,166.0,71.0,NaN,oak,"[{'id': 62, 'name': 'brioche', 'count': 32}, {...",73.0,7652.0,non_oak,"[{'id': 62, 'name': 'brioche', 'count': 32}, {...","[{'id': 434, 'name': 'vanilla', 'count': 12}, ...",69.0,5039.0,tree_fruit,"[{'id': 9, 'name': 'apple', 'count': 27}, {'id...",55.0,7200.0,NaN,microbio,"[{'id': 123, 'name': 'cream', 'count': 18}, {'...","[{'id': 417, 'name': 'toast', 'count': 16}, {'...",53.0,4488.0,70.0,butter,12.0,434.0,vanilla,12.0,79.0,caramel,12.0,292.0,oak,11.0

In [62]:
wine_taste_keyword2 = pd.json_normalize(wine_df['wine_taste_flavor_2primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword2_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword2_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_1.columns
wine_taste_keyword2_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword2_2.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_2.columns
wine_taste_keyword2_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword2_3.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_3.columns
wine_taste_keyword2_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword2_4.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_4.columns
wine_taste_keyword2_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword2_5.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword2_1, wine_taste_keyword2_2, wine_taste_keyword2_3, wine_taste_keyword2_4, wine_taste_keyword2_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_2primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r...","{'id': 50, 'seo_name': 'french-champagne', 're...",True,409.0,Champagne,,champagne,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,NaN,7747,Bollinger,bollinger,0,//images.vivino.com/wineries/backgrounds/cR1pN...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.vivino.com/winery_backgrounds/cR1pNo-...,//thumbs.

In [63]:
wine_taste_keyword3 = pd.json_normalize(wine_df['wine_taste_flavor_3primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword3_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword3_1.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_1.columns
wine_taste_keyword3_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword3_2.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_2.columns
wine_taste_keyword3_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword3_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_3.columns
wine_taste_keyword3_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword3_4.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_4.columns
wine_taste_keyword3_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword3_5.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword3_1, wine_taste_keyword3_2, wine_taste_keyword3_3, wine_taste_keyword3_4, wine_taste_keyword3_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_3primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount
0,18938,Vieilles Vignes Françaises Blanc de Noirs Brut...,vieilles-vignes-francaises-blanc-de-noirs-brut...,3,0,False,"{'id': 409, 'name': 'Champagne', 'name_en': ''...","{'id': 7747, 'name': 'Bollinger', 'seo_name': ...","{'structure': {'acidity': 4.354505, 'fizziness...","{'status': 'Normal', 'ratings_count': 1161, 'r

In [64]:
wine_taste_keyword4 = pd.json_normalize(wine_df['wine_taste_flavor_4primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword4_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword4_1.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_1.columns
wine_taste_keyword4_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword4_2.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_2.columns
wine_taste_keyword4_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword4_3.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_3.columns
wine_taste_keyword4_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword4_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_4.columns
wine_taste_keyword4_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword4_5.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_5.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword4_1, wine_taste_keyword4_2, wine_taste_keyword4_3, wine_taste_keyword4_4, wine_taste_keyword4_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_4primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_statistics

In [65]:
#Flat columns
wine_statistics = pd.json_normalize(wine_df['wine_statistics'])
wine_statistics.columns = 'wine_statistics_' + wine_statistics.columns
# print(wine_statistics.shape)
# wine_statistics

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_statistics], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 131)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_style

In [66]:
#Flat columns
wine_style = pd.json_normalize(wine_df['wine_style'])
wine_style.columns = 'wine_style_' + wine_style.columns
# print(wine_style.shape)
# wine_style

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_style], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 189)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

#### wine_style_food

In [67]:
#Flat columns
wine_food = pd.json_normalize(wine_df['wine_style_food'])

#Flat each column from food table
wine_food_1 = pd.json_normalize(wine_food[0])
wine_food_1.columns = 'wine_food_' + wine_food_1.columns
wine_food_2 = pd.json_normalize(wine_food[1])
wine_food_2.columns = 'wine_food_' + wine_food_2.columns
wine_food_3 = pd.json_normalize(wine_food[2])
wine_food_3.columns = 'wine_food_' + wine_food_3.columns
wine_food_4 = pd.json_normalize(wine_food[3])
wine_food_4.columns = 'wine_food_' + wine_food_4.columns
wine_food_5 = pd.json_normalize(wine_food[4])
wine_food_5.columns = 'wine_food_' + wine_food_5.columns


# #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_food_1, wine_food_2, wine_food_3, wine_food_4, wine_food_5], axis=1)
wine_df = wine_df.drop('wine_style_food', axis=1)
wine_df.shape

(124975, 213)

#### wine_style_grapes

In [68]:
#Flat columns
wine_grapes = pd.json_normalize(wine_df['wine_style_grapes'])

# #Flat each column from food table
wine_grapes_1 = pd.json_normalize(wine_grapes[0])
wine_grapes_1.columns = 'wine_style_grapes' + wine_grapes_1.columns
wine_grapes_2 = pd.json_normalize(wine_grapes[1])
wine_grapes_2.columns = 'wine_style_grapes_' + wine_grapes_2.columns
wine_grapes_3 = pd.json_normalize(wine_grapes[2])
wine_grapes_3.columns = 'wine_style_grapes_' + wine_grapes_3.columns
wine_grapes_4 = pd.json_normalize(wine_grapes[3])
wine_grapes_4.columns = 'wine_style_grapes_' + wine_grapes_4.columns
wine_grapes_5 = pd.json_normalize(wine_grapes[4])
wine_grapes_5.columns = 'wine_style_grapes_' + wine_grapes_5.columns
wine_grapes_6 = pd.json_normalize(wine_grapes[5])
wine_grapes_6.columns = 'wine_style_grapes_' + wine_grapes_6.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_grapes_1, wine_grapes_2, wine_grapes_3, wine_grapes_4, wine_grapes_5, wine_grapes_6], axis=1)
wine_df = wine_df.drop('wine_style_grapes', axis=1)
wine_df.shape

(124975, 242)

## Price

In [69]:
all_wines = []
prices = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        prices.append(all_wines[i][j]['price'])

In [70]:
prices_df = pd.DataFrame(prices)
prices_df.head(3)

,id,amount,discounted_from,discount_percent,type,sku,url,visibility,bottle_type_id,currency,bottle_type
0,22455852,1516.98,NaN,NaN,vc,80391,http://cultwijnen.nl/,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
1,30270704,3900.00,NaN,NaN,ppc,79231,https://www.vinsetmillesimes.com/fr/krug/79231...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
2,840861,4680.00,NaN,NaN,ppc,79865,https://www.vinsetmillesimes.com/fr/krug/79865...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."


#### Remove unnecessary columns

Price information will be used to improve recommendatio. Only the price per bottle is relevant

In [71]:
drop_lst = ['id','type','discounted_from','discount_percent','sku','url','visibility','bottle_type_id','bottle_type']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,currency
0,1516.98,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
1,3900.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
2,4680.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
3,2500.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
4,3360.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
...,...,...
124970,14.52,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
124971,12.50,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
124972,13.95,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
124973,7.20,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."


In [72]:
#Flat columns
currency = pd.json_normalize(prices_df['currency'])

# # #combine all dfs. drop original column, keep flattened
prices_df = pd.concat([prices_df, currency], axis=1)
prices_df = prices_df.drop('currency', axis=1)
prices_df.shape

# keep only currency code
drop_lst = ['name','prefix','suffix']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


,amount,code
0,1516.98,EUR
1,3900.00,EUR
2,4680.00,EUR
3,2500.00,EUR
4,3360.00,EUR
...,...,...
124970,14.52,EUR
124971,12.50,EUR
124972,13.95,EUR
124973,7.20,EUR


## Make full df from all tables above

In [73]:
all_wines_df = pd.concat([vintage_df,prices_df,statistics_df,wine_df], axis = 1)
print(all_wines_df.shape)
all_wines_df.head(3)

(124975, 250)


,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor

In [74]:
len(all_wines_df)

124975

In [75]:
columns_to_drop = ['wine_region','wine_winery','wine_taste','wine_statistics','wine_style']

In [76]:
all_wines_df = all_wines_df.drop(columns_to_drop, axis = 1)
all_wines_df

,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_winery_background_image,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscoun

In [77]:
all_wines_df.to_csv('all_wines_df_type3_v3.csv')